In [2]:
import numpy as np
import os
from scipy.io import loadmat,savemat
import hdf5storage

frame_size = 100
frame_shift = 50

genuine_features_path = 'E:\\Aditya\\CQCC\\Train'
gen1File = 'genuine_1990.mat'
#gen2File = 'genuine_2.mat'
database_path = 'E:\\Aditya\\CQCC\\overlapData_90D'
file_path = 'E:\\Aditya\\CQCC'

In [4]:
genuine1 = loadmat(os.path.join(genuine_features_path,gen1File))
keys = list(genuine1.keys())

print(keys)
position = len(keys) - 1
genuine1 = genuine1[keys[position]]
print(genuine1.shape)
print(genuine1[0].item().shape)

['__header__', '__version__', '__globals__', 'genuineFeatureCell']
(5400, 1)
(90, 357)


In [5]:
#genuine[0].item().shape[1]
# total = 0
index = 0

#frame_size = 100
#frame_shift = 10

temporary = os.path.join(file_path,'temp_overlap_90D')

if not os.path.exists(temporary):
    os.makedirs(temporary)

for i in range(genuine1.shape[0]): #genuine.shape[0]):
    featureMatrix = genuine1[i].item().T
    frames = featureMatrix.shape[0]
    k = 0
    
    context_feature = []
    diff_frame = int(np.ceil(((frames - frame_size)+1)/frame_shift))
    
    if(diff_frame == 0):
        diff_frame = 1
        
    if(diff_frame>=0):
        
        for j in range(diff_frame):
            feature = featureMatrix[k:k+frame_size]
            k = k+frame_shift
            
            if((k+frame_size)>frames):
                xtra = (k+frame_size)-frames
                padding_value = np.zeros([1,featureMatrix.shape[1]])
                feature = featureMatrix[k:frames]
                
                for pad in range(xtra):
                    feature = np.append(feature,padding_value,axis = 0)
            feature = feature.T
            context_feature.append(feature)
    else:
        context_feature = featureMatrix
        padding_frame_no = frame_size - frames
        padding_value = np.zeros([1,featureMatrix.shape[1]])
        
        for pad in range(padding_frame_no):
            context_feature = np.append(context_feature,padding_value,axis = 0)
        context_feature = context_feature.T
        context_feature = context_feature.reshape([1,context_feature.shape[0],context_feature.shape[1]])
        #print(context_feature.shape)    
    context_feature = np.array(context_feature)
#     total = total+context_feature.shape[0]
    label = np.array([1,0]).reshape([1,2])
    np.save(os.path.join(temporary,"features_"+str(index+1)), context_feature)
    np.save(os.path.join(temporary,"labels_"+str(index+1)), np.tile(label,[context_feature.shape[0],1]))
    index = index+1
    #print(context_feature.shape)
print(total)
    

NameError: name 'total' is not defined

In [3]:
total = 54000

import random as rand
randSet = rand.sample(range(1,total+1),total)
print(randSet[0:10])
if not os.path.exists(database_path):
    os.makedirs(database_path)
    
temporary = 'G:\Aditya\CQCC\\temp_overlap_90D'

[44615, 51891, 6721, 37261, 16833, 36645, 1294, 11743, 48031, 40238]


In [4]:
batch = np.zeros(shape = [2048,90,100])
batchLabel = np.zeros(shape = [2048,2])
size = 0
batchNo = 0

for i in randSet:
    feature = np.load(os.path.join(temporary,"features_"+str(i)+".npy"))
    label = np.load(os.path.join(temporary,"labels_"+str(i)+".npy"))
    #print(feature.shape)
    
    if(feature.shape[0] == 0):
        #print(i)
        continue

        
    if(size + feature.shape[0] < 2048):
        batch[size : size + feature.shape[0]] = feature
        batchLabel[size : size + feature.shape[0]] = label
        
        size = size+feature.shape[0]
    
    elif(size+feature.shape[0] >= 2048):
        batch[size : ] = feature[ : 2048-size]
        batchLabel[size : ] = label[ : 2048-size]
        
        batchNo = batchNo + 1
        #print(batchNo)
        
        np.save(os.path.join(database_path,"batch_"+str(batchNo)),batch)
        np.save(os.path.join(database_path,"label_"+str(batchNo)),batchLabel)
        
        batch = np.zeros(shape=[2048,90,100])
        batchLabel = np.zeros(shape = [2048,2])
            
        batch[: size+feature.shape[0] - 2048] = feature[ 2048 - size: ]
        batchLabel[ : size+feature.shape[0] - 2048] = label[ 2048 - size : ]
            
        size = size+feature.shape[0] - 2048
            
print(batchNo)

if(size>0):
    np.save(os.path.join(database_path,"batch_"+str(batchNo)), batch)
    np.save(os.path.join(database_path,"label_"+str(batchNo)),batchLabel)

220
